In [1]:
import json
import pandas as pd
from glom import glom
import seaborn as sns

In [110]:
decay = 0.8

In [128]:
with open('../data/2018ish_data.json') as json_file:
    raw = pd.read_json(json_file)
    
raw

,course_number,course_name,term,year,eligible,responded,teachers,ratings,inclass_hours,outclass_hours
0,1.74,"Land, Water, Food, and Climate",Fall,2020,11,5,"[{'id': '900026886', 'name': 'McLaughlin, Denn...",{'The expectations for the subject were clearl...,{},{}
1,STS.036,Science in American Life: 1920-2020,Fall,2021,10,8,"[{'id': '923680689', 'name': 'Durant, John R.'...",{'Subject expectations were clearly defined': ...,"{'avg': 3.6, 'responses': 8.0, 'median': 3.5, ...","{'avg': 3.6, 'responses': 8.0, 'median': 3.5, ..."
2,1.008,Engineering Solutions to Societal Challenges,Fall,2020,18,9,"[{'id': '923797094', 'name': 'Amin, Saurabh', ...",{'The expectations for the subject were clearl...,{},{}
3,1.38,Engineering Geology,Fall,2020,2,1,"[{'id': '900012303', 'name': 'Einstein, Herber...",{'The expectations for the subject were clearl...,{},{}
4,1.000,Introduction to Computer Programming and Numer...,Fall,2020,13,9,"[{'id': '922303500', 'name': 'Juanes, Ruben', ...",{'The expectations for the subject were clearl...,{},{}
...,...,...,...,...,...,...,...,...,...,...
5446,8.01L,Physics I,Fall,2018,66,39,"[{'id': '921444924', 'name': 'Jarillo-Herrero,...",{'Subject expectations were clearly defined': ...,"{'avg': 7.4, 'responses': 37.0, 'median': 7.0,...","{'avg': 7.4, 'responses': 37.0, 'median': 7.0,..."
5447,8.286,The Early Universe,Fall,2018,39,23,"[{'id': '900020325', 'name': 'Guth, Alan', 'ty...",{'Subject expectations were clearly defined': ...,"{'avg': 5.9, 'responses': 22.0, 'median': 6.0,...","{'avg': 5.9, 'responses': 22.0, 'median': 6.0,..."
5448,8.033,Relativity,Fall,2018,69,52,"[{'id': '921587983', 'name': 'Vitale, Salvator...",{'Subject expectations were clearly defined': ...,"{'avg': 7.1, 'responses': 50.0, 'median': 7.0,...","{'avg': 7.1, 'responses': 50.0, 'median': 7.0,..."
5449,8.225,"Einstein, Oppenheimer, Feynman: Physics in the...",Fall,2018,50,34,"[{'id': '900051737', 'name': 'Kaiser, David I....",{'Subject expectations were clearly defined': ...,"{'avg': 4.9, 'responses': 33.0, 'median': 5.0,...","{'avg': 4.9, 'responses': 33.0, 'median': 5.0,..."


In [129]:
class_df = raw
class_df['rating'] = raw['ratings'].apply(lambda row: glom(row, 'Overall rating of the subject.avg', default=None))
class_df['hours'] = pd.to_numeric(raw['inclass_hours'].apply(lambda row: glom(row, 'avg', default=None)))
class_df.dropna(subset=['rating', 'year'], inplace=True)
# display(pd.to_numeric(class_df['year'], errors='coerce'))
class_df['year'] = pd.to_numeric(class_df['year'], errors='coerce', downcast  = 'integer')
# class_df = class_df[~(class_df['term'] == 'IAP')]
class_df['time'] = [2022 for _ in range(len(class_df))] - class_df['year']
class_df['time'] = 2 * class_df['time'] + (~class_df['term'].str.contains('Fall')).astype('int') 
class_df['weight'] = class_df['responded'] * class_df['time'].rpow(decay)

class_df = class_df.drop_duplicates(subset=['term', 'year', 'eligible', 'responded', 'hours', 'time', 'rating'])

class_df = class_df[['course_number', 'course_name', 'responded', 'rating', 'hours', 'time', 'weight']]

class_df

,course_number,course_name,responded,rating,hours,time,weight
0,STS.036,Science in American Life: 1920-2020,8,6.6,3.6,2.0,5.120000
1,4.181,Architectural Design Workshop,0,0.0,0.0,5.0,0.000000
2,4.182,Architectural Design Workshop,4,7.0,7.7,5.0,1.310720
3,1.091,Traveling Research Environmental eXperience (T...,5,6.4,26.2,5.0,1.638400
4,4.02A,Design Studio: How to Design Intensive,11,6.4,5.5,5.0,3.604480
...,...,...,...,...,...,...,...
2833,8.13,Experimental Physics I,26,5.4,15.3,8.0,4.362076
2834,8.01L,Physics I,39,5.2,7.4,8.0,6.543114
2835,8.286,The Early Universe,23,6.7,5.9,8.0,3.858760
2836,8.033,Relativity,52,5.7,7.1,8.0,8.724152


In [144]:
filt = class_df[~class_df['course_number'].str.contains('21G')]
filt = filt[filt['weight'] != 0]
filt.reset_index(drop = True, inplace = True)
filt.dropna(inplace = True)
# filt[filt.isnull().values]
# filt

In [146]:
m = sum(filt['rating'] * filt['weight']) / sum(filt['weight'])
m

5.807321775113703

In [148]:
c = filt.sort_values('weight').iloc[[len(filt) / 4]]['weight'].iloc[0]
c

1.0485760000000004

In [181]:
filt['weighted_rating'] = filt['weight'] * filt['rating']
grouped = filt.groupby('course_number')
df = grouped.mean()
df.drop(columns = ['time', 'rating', 'adjusted_rating', 'responded'], inplace=True)
df['weight'] = grouped.sum()['weight']
df['weighted_rating'] = grouped.sum()['weighted_rating']
df.reset_index(inplace=True)
filt_names = filt[['course_name', 'course_number']]
df = df.merge(filt_names, how = 'inner', left_on = 'course_number', right_on = 'course_number')
df.drop_duplicates(inplace=True)

In [182]:
df['adjusted_rating'] = (df['weighted_rating'] + c * m) / (df['weight'] + c)
df.sort_values('adjusted_rating', ascending = False)[:40]

,course_number,hours,weight,weighted_rating,course_name,adjusted_rating
1353,21M.460,0.650000,11.114906,77.217137,MIT Senegalese Drum Ensemble,6.848907
1271,21L.591,12.700000,6.894387,48.260710,Literary London,6.842551
640,15.6731,4.300000,6.710886,46.976205,Negotiation Analysis,6.838827
195,11.011,7.700000,13.893632,95.997133,The Art and Science of Negotiation,6.832093
639,15.672,3.700000,15.400960,106.266624,Negotiation Analysis,6.830347
200,11.111,7.300000,5.033165,35.232154,Leadership in Negotiation: Advanced Applications,6.794366
500,15.268,3.200000,4.823450,33.764147,Choice Points: Thinking about Life and Leaders...,6.787022
795,17.317,3.500000,8.388608,57.881395,US Social Policy,6.778591
1425,21W.762,3.200000,5.557453,38.640026,Poetry Workshop,6.771003
2081,6.809,7.933333,14.428406,98.448703,Interactive Music Systems,6.754426


**todo:**
- penalize standard deviation ? idk if we care
- make a professor df and do calculations on them